In [1]:
#Import needed modules
import pandas as pd
import pymarc
import re
from pymarc import MARCWriter 
from pymarc import MARCReader
from pymarc import Record
from pymarc.field import Field
import csv

In [2]:
#Create pandas dataframe
MasterChart = pd.read_csv('MasterChartcleaner.csv')
#Index the Publication_Place column for various MARC writer options below.
idx = pd.Index(MasterChart.Publication_Place)

print(idx)

Index([''s Graven-Hage', 'A Aix', 'A Amsterdam', 'A Amsterdam ',
       'A Amsterdam.', 'A Anvers', 'A Avignon', 'A Basle', 'A Berlin',
       'A Bésiers',
       ...
       'Zaragoza', 'Zerbst', 'Zu Basel', 'Zullichouiae', 'Zurich', 'Zürich',
       'Zürych', 'Zutphaniae', 'Zwollae', 'Zwolle'],
      dtype='object', name='Publication_Place', length=2227)


In [3]:
#Creates CSV filewriter for writing Country Code, 260/264, 752, and 001 for human review
with open('ReportMARCwith752s.csv', 'a') as csvfile:
    goodfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
with open('ReportMARCalreadywith752s.csv', 'a') as csvfile:
    okayishfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
with open('ReportMARCwithno752s.csv', 'a') as csvfile:
    badfilewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)

In [4]:
#Create reader and writer
reader = MARCReader(open('Test-27-Records.mrc', 'rb'))
writergood = MARCWriter(open('MARCwith752s.mrc', 'wb'))
writerokayish = MARCWriter(open('MARCalreadywith752s.mrc', 'wb'))
writerbad = MARCWriter(open('MARCwithno752s.mrc', 'wb'))

#Create some variables
field008 = "No 008"
field260a = "No 260"
clean260a = "No 260"
field264a = "No 264"
clean264a = "No 264"
field752a = "no value"
field752b = "no value"
field752d = "no value"
field752e = "no value"
field7524 = "no value"
field7521 = "no value"
field533counter = 0
field260and264counter = 0
field260acounter = 0
field260ecounter = 0
field264acounter = 0
message = "No match found"



In [5]:
#Start loop through the records in the list.
for record in reader:
    #Grab 008 from record and assign to variable.
    field008 = record['008']
    #Grab MARC Country Code and assign to variable and remove any whitespace for 2 character-only codes.
    countrycode = field008.data[15:18].rstrip(' ')
    #Check to see if there is already a 752 in the record.
    if record['752'] is None:
        if record['533'] is None:
            field260and264counter = record.get_fields('260','264')
            if len(field260and264counter) == 1:
            #Check to see if there is a 260.
                if record['260'] is not None:
                    #Check to see how many a subfields there are
                    field260acounter = record['260'].get_subfields('a')
                    field260ecounter = record['260'].get_subfields('e')
                    field260aecounter = record['260'].get_subfields('a') + record['260'].get_subfields('e')
                    print(field260aecounter)
                    if len(field260aecounter) == 1:
                        #Capture the transcribed publication location and clean off ISBD and other punctuation in preparation for matching. 
                        field260a = record['260']['a'].rstrip(' :')
                        clean260a = result = re.sub('[\]\[\)\(\;\,\:\?]','',field260a)
                        #Assign 752 subfields based off of 260 value.
                        field752e = "publication place."
                        field7524 = "http://id.loc.gov/vocabulary/relators/pup.html"
                         #This Filters the original MasterChart dataframe for MARC Country Code and imprint location captured from the bibliographic record.
                        QueriedChart = MasterChart.query('CountryCode == @countrycode and Publication_Place == @clean260a', inplace = False)
                        #This returns the size of the QueriedChart, incase there aren't any matches.
                        size = QueriedChart.size 
                        #Used to send records to the Badwriter if False; Goodwriter if True.
                        IndexChecker = idx.contains(clean260a)
                        #if there is more or less than 1 subfield a, send it to the Badwriter.
                    else:
                        IndexChecker = False
                        message = "More than one 260 subfield a/e is present in record.  Resolve manually."
                elif record['264'] is not None:
                    #Check to see how many $a subfields there are
                    field264acounter = record['264'].get_subfields('a')
                    if len(field264acounter) == 1:
                        field264a = record['264']['a'].rstrip(' :')
                        clean264a = result = re.sub('[\]\[\)\(\;\,\:\?]','',field264a)
                        #This Filters the original MasterChart dataframe for MARC Country Code and Publication Place
                        QueriedChart = MasterChart.query('CountryCode == @countrycode and Publication_Place == @clean264a', inplace = False)
                        #This returns the size of the QueriedChart, incase there aren't any matches.
                        size = QueriedChart.size 
                        #Used to send records to the Badwriter if False; Goodwriter if True.
                        IndexChecker = idx.contains(clean264a)
                    #if there is more or less than 1 subfield a, send it to the Badwriter.
                    else:
                        IndexChecker = False
                        message = "More or less than one 264 subfield a is present in record.  Resolve manually."
                    #Assign value to 752 subfield e and subfield 4 based on 264 indicators.
                    for imprintrole in record.get_fields('264'):
                        #Assign 752 subfield e and 4 for production place
                        if imprintrole.indicator2 == '0':
                            field752e = "production place."
                            field7524 = "http://id.loc.gov/vocabulary/relators/prp.html"
                        #Assign 752 subfield e and 4 for publication place    
                        elif imprintrole.indicator2 == '1':
                            field752e = "publication place."
                            field7524 = "http://id.loc.gov/vocabulary/relators/pup.html"
                        #Assign 752 subfield e and 4 for distribution place    
                        elif imprintroleindicator2 == '2':
                            field752e = "distribution place."
                            field7524 = "http://id.loc.gov/vocabulary/relators/dbp.html"
                        #Assign 752 subfield e and 4 for manufacture place  
                        elif imprintroleindicator2 == '3':
                            field752e = "manufacture place."
                            field7524 = "http://id.loc.gov/vocabulary/relators/mfp.html"
                        #Handle records that do not have indicators with 264, unlikely, but possible.   
                        else:  
                            #Sends problem records with 264s with no indicators to the Badwriter.
                            IndexChecker = False
                            message = "264 field is lacking indicators.  Resolve manually."
                #Sends problem records with no 260/264 at all to the Badwriter.            
                else:  
                    IndexChecker = False
                    message = "260/264 fields are not present in record.  Resolve manually."
            #if there is more or less than one 260/264 tag combo, send it to the Badwriter.
            else:
                IndexChecker = False
                message = "More than one 260/264 fields are present in record.  Resolve manually."
        #if there is a 533 in the record, send to the Badwriter for manual review.
        else:  
            IndexChecker = False
            message = "A 533 is present in record.  Resolve manually."  
                
        #If the cleaned-up versions of the 260/264 values were found using the IndexChecker, aka the IndexChecker is True.
        if IndexChecker is True and size > 0:
            #Assign the value of Term3 which gives the index of the row in the MasterChart.
            Term3 = QueriedChart.index[0]
            #Assign the values of the 752 subfields that weren't already assigned above. 
            field752a = MasterChart.loc[Term3,"752a"]
            field752b = MasterChart.loc[Term3,"752b"]
            field752d = MasterChart.loc[Term3,"752d"]
            field7521 = MasterChart.loc[Term3,"Geonames_URI"]
            #Add the assembled 752 subfields to the MARC Record.   
            record.add_ordered_field(
                Field(
                tag = '752',
                indicators = [' ',' '],
                subfields = [
                    'a', field752a,
                    'b', field752b,
                    'd', field752d,
                    'e', field752e,
                    '4', field7524,
                    '1', field7521,
                        ]))
                #remove any subfields with "no value" as a subfield value.
            if field752a == 'no value':
                record['752'].delete_subfield('a')
            else:
                print(record['752'])
            if field752b == 'no value':
                record['752'].delete_subfield('b')
            else:
                print(record['752'])
            if field752d == 'no value':
                record['752'].delete_subfield('d')
            else:
                print(record['752'])
            if field752e == 'no value':
                record['752'].delete_subfield('e')
            else:
                print(record['752'])
            if field7521 == 'no value':
                record['752'].delete_subfield('1')
            else:
                print(record['752'])
            #The MARC record is written to the Goodwriter after the 752 is populated and added to the record.  
            #Create dictionary of values for Goodwriter CSV file
            MMSID = [record['001']]
            OCLCno = [record['035']]
            ccfrom008 = countrycode
            field260 = [record['260']]
            field264 = [record['264']]
            field752 = [record['752']]
            #Create a dictionary of lists 
            dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752}
            gooddf = pd.DataFrame(dict) 
            #Saves MARC information to the good dataframe 
            gooddf.to_csv('ReportMarcwith752s.csv', mode='a', header=False) 
            writergood.write(record)          
            #If the cleaned-up versions of the 260/264 values were NOT found using the IndexChecker, aka the IndexChecker is False.
            #The MARC record is written to the Badwriter for manual review and evaluation.
        else:
            #Create dictionary of values for Badwriter CSV file
            MMSID = [record['001']]
            OCLCno = [record['035']]
            ccfrom008 = countrycode
            field260 = [record['260']] 
            field264 = [record['264']]
            field752 = [record['752']]
            
            
            #Create a dictionary of lists 
            dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752, 'Error': message}
            baddf = pd.DataFrame(dict) 
            #Saves MARC information to the bad dataframe 
            baddf.to_csv('ReportMARCwithno752s.csv', mode='a', header=False)  
            writerbad.write(record)        
    #If there is already a 752 in the record, send to the Okayish writer for manual review.
    else:
        #enrich 752 with URIS?
        #Create dictionary of values for Okayish CSV file 
        MMSID = [record['001']]
        OCLCno = [record['035']]
        ccfrom008 = countrycode
        field260 = [record['260']] 
        field264 = [record['264']]
        field752 = [record['752']]
        #Create a dictionary of lists 
        dict = {'Alma MMS ID': MMSID, 'OCLC number': OCLCno, 'Country Code': ccfrom008, '260': field260, '264': field264, '752': field752}
        okayishdf = pd.DataFrame(dict) 
        #Saves MARC information to the okayish dataframe 
        okayishdf.to_csv('ReportMARCalreadywith752s.csv', mode='a', header=False) 
        writerokayish.write(record)
#Close writers                   
writergood.close()      
writerbad.close()
writerokayish.close()

['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http:

['Königsberg,', 'Leipzig :']
['Paris :']
=752  \\$aFrance$bno value$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=7

['Berolini :']
=752  \\$aGermany$bno value$dBerlin,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2950159
=752  \\$aGermany$dBerlin,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2950159
=752  \\$aGermany$dBerlin,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2950159
=752  \\$aGermany$dBerlin,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2950159
['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Brita

['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
['Leipzig,']
=752  \\$aGermany$bno value$dLeipzig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2879139
=752  \\$aGermany$dLeipzig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org

=752  \\$aGreat Britain$bScotland$dGlasgow,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2648579
=752  \\$aGreat Britain$bScotland$dGlasgow,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2648579
=752  \\$aGreat Britain$bScotland$dGlasgow,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2648579
=752  \\$aGreat Britain$bScotland$dGlasgow,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2648579
=752  \\$aGreat Britain$bScotland$dGlasgow,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2648579
['In Firenze :']
=752  \\$aItaly$bno value$dFlorence,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3176959
=752  \\$aItaly$dFlorence,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/

...

['Bruxelles :']
=752  \\$aBelgium$bno value$dBrussels,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2800866
=752  \\$aBelgium$dBrussels,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2800866
=752  \\$aBelgium$dBrussels,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2800866
=752  \\$aBelgium$dBrussels,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2800866
['Leidae Batavorum :']
=752  \\$aNetherlands$bno value$dLeiden,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2751773
=752  \\$aNetherlands$dLeiden,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2751773
=752  \\$aNetherlands$dLeiden,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2751773
=752  \\$aNetherlands$dLe

['London :']
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
=752  \\$aGreat Britain$bEngland$dLondon,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2643743
['Leipzig :']
=752  \\$aGermany$bno value$dLeipzig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2879139
=752  \\$aGermany$dLeipzig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.or

=752  \\$aGermany$bno value$dBraunschweig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2945024
=752  \\$aGermany$dBraunschweig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2945024
=752  \\$aGermany$dBraunschweig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2945024
=752  \\$aGermany$dBraunschweig,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2945024
['Parma :']
=752  \\$aItaly$bno value$dParma,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3171457
=752  \\$aItaly$dParma,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3171457
=752  \\$aItaly$dParma,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/3171457
=752  \\$aItaly$dParma,$epublication place.$4http://id.lo

['Á Paris :']
=752  \\$aFrance$bno value$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
['A Paris :']
=752  \\$aFrance$bno value$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocabulary/relators/pup.html$1http://sws.geonames.org/2988507
=752  \\$aFrance$dParis,$epublication place.$4http://id.loc.gov/vocab

NameError: name 'Message' is not defined